In [11]:
using PyCall, PyPlot, JuMP
using MARGO

In [5]:
] add Combinatorics

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %  Updating registry at `~/.julia/registries/MimiRegistry`
  Updating git-repo `https://github.com/mimiframework/MimiRegistry.git`
 Resolving package versions...
  Updating `~/Documents/code/MARGO.jl/Project.toml`
  [861a8166] + Combinatorics v1.0.2
  Updating `~/Documents/code/MARGO.jl/Manifest.toml`
  [861a8166] + Combinatorics v1.0.2


In [6]:
using Combinatorics

┌ Info: Precompiling Combinatorics [861a8166-3701-5b0c-9a16-15d98fcdc6aa]
└ @ Base loading.jl:1273


# Optimizating the Global Climate Action Policy Portfolio

In [7]:
model_og = ClimateModel();
max_deployment_og = Dict("mitigate"=>1., "remove"=>1., "geoeng"=>1., "adapt"=>0.4)

Dict{String,Float64} with 4 entries:
  "geoeng"   => 1.0
  "mitigate" => 1.0
  "remove"   => 1.0
  "adapt"    => 0.4

In [8]:
controls = collect(keys(max_deployment_og));

In [9]:
control_permutations = combinations(controls) |> collect

15-element Array{Array{String,1},1}:
 ["geoeng"]                               
 ["mitigate"]                             
 ["remove"]                               
 ["adapt"]                                
 ["geoeng", "mitigate"]                   
 ["geoeng", "remove"]                     
 ["geoeng", "adapt"]                      
 ["mitigate", "remove"]                   
 ["mitigate", "adapt"]                    
 ["remove", "adapt"]                      
 ["geoeng", "mitigate", "remove"]         
 ["geoeng", "mitigate", "adapt"]          
 ["geoeng", "remove", "adapt"]            
 ["mitigate", "remove", "adapt"]          
 ["geoeng", "mitigate", "remove", "adapt"]

In [12]:
results = Dict()

for control_permutation in control_permutations
    model = ClimateModel();
    
    # Assert that some controls not be deployed
    max_deployment = copy(max_deployment_og)
    for control in controls
        if ~(control in control_permutation)
            max_deployment[control] = 0.
            # Override initial condition constraints
            if ~isnothing(getfield(model.economics, Symbol(string(control,"_init"))))
                setfield!(model.economics, Symbol(string(control,"_init")), 0.)
            end
        end
    end
    @time m = optimize_controls!(model, max_deployment = max_deployment, temp_goal=2.0);
    
    results[control_permutation] = Dict("model" => model, "NPV" => MARGO.discounted_total_control_cost(model), "status" => raw_status(m))
    
end

Solve_Succeeded
  0.085605 seconds (320.22 k allocations: 8.069 MiB, 15.98% gc time)
Solve_Succeeded
  0.314386 seconds (827.60 k allocations: 16.110 MiB)
Infeasible_Problem_Detected
  0.162303 seconds (300.15 k allocations: 7.491 MiB, 9.50% gc time)
Infeasible_Problem_Detected
  6.475427 seconds (18.66 M allocations: 305.089 MiB, 0.65% gc time)
Solve_Succeeded
  0.042581 seconds (96.16 k allocations: 4.142 MiB)
Solve_Succeeded
  0.026941 seconds (75.15 k allocations: 3.760 MiB)
Solve_Succeeded
  0.038576 seconds (102.98 k allocations: 4.257 MiB)
Solve_Succeeded
  0.048821 seconds (113.05 k allocations: 4.447 MiB)
Solve_Succeeded
  0.048205 seconds (116.69 k allocations: 4.516 MiB)
Solve_Succeeded
  0.062988 seconds (173.17 k allocations: 5.471 MiB)
Solve_Succeeded
  0.040926 seconds (101.26 k allocations: 4.280 MiB)
Solve_Succeeded
  0.039301 seconds (101.72 k allocations: 4.291 MiB)
Solve_Succeeded
  0.056818 seconds (101.40 k allocations: 4.290 MiB, 23.73% gc time)
Solve_Succeeded
 

In [13]:
min_value = Inf
for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded") & (value["NPV"] < min_value)
        min_value = value["NPV"]
    end
end

for (index, value) in pairs(results)
    if (value["status"] == "Solve_Succeeded")
        abs_value = Int64(round((value["NPV"])))
        rel_value = Int64(round((value["NPV"]/min_value - 1.)*100))
        print("$index, $abs_value, $rel_value%\n")
    else
        print("$index has no solution.\n")
    end
end


["geoeng", "mitigate", "remove"], 143, 5%
["remove"] has no solution.
["mitigate", "remove"], 188, 38%
["mitigate"], 322, 137%
["geoeng", "mitigate", "remove", "adapt"], 136, 0%
["geoeng", "adapt"], 304, 124%
["geoeng", "mitigate", "adapt"], 181, 33%
["mitigate", "remove", "adapt"], 178, 31%
["geoeng"], 412, 204%
["geoeng", "remove", "adapt"], 221, 63%
["mitigate", "adapt"], 277, 104%
["geoeng", "mitigate"], 199, 47%
["remove", "adapt"], 485, 257%
["geoeng", "remove"], 268, 97%
["adapt"] has no solution.
